In [10]:
from transformers import CLIPVisionModel
from datasets import load_dataset, Image, load_dataset_builder

In [2]:
class Retriever:
    def __init__(self, dataset_path="dalle-mini/open-images"):
        self.dataset = load_dataset(dataset_path)

In [18]:
dataset = load_dataset("dalle-mini/open-images", split="train")

Using custom data configuration dalle-mini--open-images-b8027789dd5454d5


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

OSError: [Errno 22] Invalid argument

In [4]:
dataset

NameError: name 'dataset' is not defined